In [4]:
#### Install necessary packages

!pip install crosslingual-coreference==0.2.3 spacy-transformers==1.1.5 wikipedia neo4j
!pip install --upgrade google-cloud-storage
!pip install transformers==4.18.0
!python -m spacy download en_core_web_sm
!pip install py2neo

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 51 kB 181 kB/s 
     |████████████████████████████████| 173 kB 30.6 MB/s 
     |██████████████████████████████▎ | 834.1 MB 1.2 MB/s eta 0:00:41tcmalloc: large alloc 1147494400 bytes == 0x2360000 @  0x7f4ba852b615 0x58ead6 0x4f355e 0x4d222f 0x51041f 0x5b4ee6 0x58ff2e 0x510325 0x5b4ee6 0x58ff2e 0x50d482 0x4d00fb 0x50cb8d 0x4d00fb 0x50cb8d 0x4d00fb 0x50cb8d 0x4bac0a 0x538a76 0x590ae5 0x510280 0x5b4ee6 0x58ff2e 0x50d482 0x5b4ee6 0x58ff2e 0x50c4fc 0x58fd37 0x50ca37 0x5b4ee6 0x58ff2e
     |████████████████████████████████| 881.9 MB 1.9 kB/s 
     |████████████████████████████████| 23.2 MB 1.2 MB/s 
     |████████████████████████████████| 2.9 MB 57.2 MB/s 
     |████████████████████████████████| 730 kB 60.5 MB/s 
     |████████████████████████████████| 464 kB 67.3 MB/s 
     |████████████████████████████████| 3.8 MB 44.0 MB/s 
     |███████████████████████

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.0 MB 14.3 MB/s 
     |████████████████████████████████| 6.6 MB 49.9 MB/s 
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.13.1
    Uninstalling tokenizers-0.13.1:
      Successfully uninstalled tokenizers-0.13.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.17.0
    Uninstalling transformers-4.17.0:
      Successfully uninstalled transformers-4.17.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spacy-transformers 1.1.5 requires transformers<4.18.0,>=3.4.0, but you have transformers 4.18.0 which is incompatible.
Looking in indexes: https://pypi.org/simpl

In [9]:
# Add rebel component https://github.com/Babelscape/rebel/blob/main/spacy_component.py
import requests
import re
import hashlib
import json
import csv
import io
import torch
import spacy
import crosslingual_coreference
import pandas as pd
import wikipedia
from spacy import Language
from spacy.tokens import Doc, Span
from spacy.matcher import PhraseMatcher
from typing import List
from py2neo import Graph
from neo4j import GraphDatabase, basic_auth
from transformers import pipeline

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


ImportError: ignored

In [ ]:

def call_wiki_api(item):
  try:
    url = f"https://www.wikidata.org/w/api.php?action=wbsearchentities&search={item}&language=en&format=json"
    data = requests.get(url).json()
    # Return the first id (Could upgrade this in the future)
    return data['search'][0]['id']
  except:
    return 'id-less'

def extract_triplets(text):
    """
    Function to parse the generated text and extract the triplets
    """
    triplets = []
    relation, subject, relation, object_ = '', '', '', ''
    text = text.strip() #remove spaces at beginning and end
    current = 'x'
    for token in text.replace("<s>", "").replace("<pad>", "").replace("</s>", "").split():
        if token == "<triplet>":
            current = 't'
            if relation != '':
                triplets.append({'head': subject.strip(), 'type': relation.strip(),'tail': object_.strip()})
                relation = ''
            subject = ''
        elif token == "<subj>":
            current = 's'
            if relation != '':
                triplets.append({'head': subject.strip(), 'type': relation.strip(),'tail': object_.strip()})
            object_ = ''
        elif token == "<obj>":
            current = 'o'
            relation = ''
        else:
            if current == 't':
                subject += ' ' + token
            elif current == 's':
                object_ += ' ' + token
            elif current == 'o':
                relation += ' ' + token
    if subject != '' and relation != '' and object_ != '':
        triplets.append({'head': subject.strip(), 'type': relation.strip(),'tail': object_.strip()})

    return triplets


@Language.factory(
    "rebel",
    requires=["doc.sents"],
    assigns=["doc._.rel"],
    default_config={
        "model_name": "Babelscape/rebel-large",
        "device": 0,
    },
)
class RebelComponent:
    def __init__(
        self,
        nlp,
        name,
        model_name: str,
        device: int,
    ):
        assert model_name is not None, ""
        self.triplet_extractor = pipeline("text2text-generation", model=model_name, tokenizer=model_name, device=device)
        self.entity_mapping = {}
        # Register custom extension on the Doc
        if not Doc.has_extension("rel"):
          Doc.set_extension("rel", default={})

    def get_wiki_id(self, item: str):
        mapping = self.entity_mapping.get(item)
        if mapping:
          return mapping
        else:
          res = call_wiki_api(item)
          self.entity_mapping[item] = res
          return res

    
    def _generate_triplets(self, sent: Span) -> List[dict]:
          output_ids = self.triplet_extractor(sent.text, return_tensors=True, return_text=False)[0]["generated_token_ids"]["output_ids"]
          extracted_text = self.triplet_extractor.tokenizer.batch_decode(output_ids[0])
          extracted_triplets = extract_triplets(extracted_text[0])
          return extracted_triplets

    def set_annotations(self, doc: Doc, triplets: List[dict]):
        for triplet in triplets:

            # Remove self-loops (relationships that start and end at the entity)
            if triplet['head'] == triplet['tail']:
                continue

            # Use regex to search for entities
            head_span = re.search(triplet["head"], doc.text)
            tail_span = re.search(triplet["tail"], doc.text)

            # Skip the relation if both head and tail entities are not present in the text
            # Sometimes the Rebel model hallucinates some entities
            if not head_span or not tail_span:
              continue

            index = hashlib.sha1("".join([triplet['head'], triplet['tail'], triplet['type']]).encode('utf-8')).hexdigest()
            if index not in doc._.rel:
                # Get wiki ids and store results
                doc._.rel[index] = {"relation": triplet["type"], "head_span": {'text': triplet['head'], 'id': self.get_wiki_id(triplet['head'])}, "tail_span": {'text': triplet['tail'], 'id': self.get_wiki_id(triplet['tail'])}}

    def __call__(self, doc: Doc) -> Doc:
        for sent in doc.sents:
            sentence_triplets = self._generate_triplets(sent)
            self.set_annotations(doc, sentence_triplets)
        return doc

In [ ]:

import torch
if torch.cuda.is_available():
    !nvidia-smi
else:
    print("☹️")

In [ ]:
DEVICE = -1 # Number of the GPU, -1 if want to use CPU

# Add coreference resolution model
coref = spacy.load('en_core_web_sm', disable=['ner', 'tagger', 'parser', 'attribute_ruler', 'lemmatizer'])
coref.add_pipe(
    "xx_coref", config={"chunk_size": 2500, "chunk_overlap": 2, "device": DEVICE})


# Define rel extraction model
rel_ext = spacy.load('en_core_web_sm', disable=['ner', 'lemmatizer', 'attribute_rules', 'tagger'])
rel_ext.add_pipe("rebel", config={
    'device':DEVICE, # Number of the GPU, -1 if want to use CPU
    'model_name':'Babelscape/rebel-large'} # Model used, will default to 'Babelscape/rebel-large' if not given
    )

In [ ]:
#### Test Input & Output
input_text = "Amyotrophic lateral sclerosis (ALS) is a progressive neurodegenerative disease that affects motor neurons. Mutations in the SPTLC1 subunit of serine palmitoyltransferase (SPT), which catalyzes the first step in the de novo synthesis of sphingolipids (SLs), cause childhood-onset ALS. SPTLC1-ALS variants map to a transmembrane domain that interacts with ORMDL proteins, negative regulators of SPT activity."

#nlp = spacy.load("en_core_web_sm")
matcher = PhraseMatcher(coref.vocab)
terms = ["SPTLC1", "ALS", "ORMDL"]
# Only run nlp.make_doc to speed things up
patterns = [coref.make_doc(text) for text in terms]
matcher.add("TerminologyList", patterns)

coref_text = coref(input_text)._.resolved_text


matches = matcher(doc)

for match_id, start, end in matches:
    span = doc[start:end]
    print(span.text)


doc = rel_ext(coref_text)

for value, rel_dict in doc._.rel.items():
    print(f"{value}: {rel_dict}")

In [ ]:
Inputfile = "./pmid"
Format = "pubtator"
Bioconcept = ""

In [ ]:
#### Extract & Store the triplets

def SubmitPMIDList(Pmid,Format,Bioconcept):
    
    json = {}

    with io.open(Inputfile,'r',encoding="utf-8") as file_input:
        json = {"pmids": [pmid.strip() for pmid in file_input.readlines()]}
    
    if Bioconcept != "": 
        json["concepts"] = Bioconcept.split(",")

    r = requests.post("https://www.ncbi.nlm.nih.gov/research/pubtator-api/publications/export/" + Format , json = json)

    data = r.text.splitlines()
    result = data
    title = data[0][11:]
    abstract = data[1][11:]

    result = title + abstract

    if r.status_code != 200 :
        print ("[Error]: HTTP code "+ str(r.status_code))
    else:
        print(result)
        return result

def store_pubtator_summary(i):
  try:
    input_text = SubmitPMIDList([i], Format, Bioconcept)
    coref_text = coref(input_text)._.resolved_text
    doc = rel_ext(coref_text)
    params = [rel_dict for value, rel_dict in doc._.rel.items()]
    run_query(import_query, {'data': params})
  except Exception as e:
    print(f"Couldn't parse text for {i} due to {e}")

def run_query(query, params={}):
    with driver.session() as session:
        result = session.run(query, params)
        return pd.DataFrame([r.values() for r in result], columns=result.keys())

with io.open(Inputfile,'r',encoding="utf-8") as file_input:
  pmidlist = {"pmids": [pmid.strip() for pmid in file_input.readlines()]}

for i in pmidlist['pmids']:
  store_pubtator_summary(i)

SPTLC1 variants associated with ALS produce distinct sphingolipid signatures through impaired interaction with ORMDL proteins.Amyotrophic lateral sclerosis (ALS) is a progressive neurodegenerative disease affecting motor neurons. Mutations in the SPTLC1 subunit of serine-palmitoyltransferase (SPT), which catalyzes the first step in the de novo synthesis of sphingolipids cause childhood-onset ALS. SPTLC1-ALS variants map to a transmembrane domain that interacts with ORMDL proteins, negative regulators of SPT activity. We show that ORMDL binding to the holoenzyme complex is impaired in cells expressing pathogenic SPTLC1-ALS alleles, resulting in increased sphingolipid synthesis and a distinct lipid signature. C-terminal SPTLC1 variants cause the peripheral sensory neuropathy HSAN1 due to the synthesis of 1-deoxysphingolipids (1-deoxySLs) that form when SPT metabolizes L-alanine instead of L-serine. Limiting L-serine availability in SPTLC1-ALS expressing cells increased 1-deoxySL and shif

In [ ]:
#### Send to Neo4j

host = 'bolt://3.87.24.194:7687' ## CHANGES WHEN NEW SERVER IS UP
user = 'neo4j'
password = 'applicant-electron-net' ## CHANGES WHEN NEW SERVER IS UP
driver = GraphDatabase.driver(host,auth=(user, password))

import_query = """
UNWIND $data AS row
MERGE (h:Entity {id: CASE WHEN NOT row.head_span.id = 'id-less' THEN row.head_span.id ELSE row.head_span.text END})
ON CREATE SET h.text = row.head_span.text
MERGE (t:Entity {id: CASE WHEN NOT row.tail_span.id = 'id-less' THEN row.tail_span.id ELSE row.tail_span.text END})
ON CREATE SET t.text = row.tail_span.text
WITH row, h, t
CALL apoc.merge.relationship(h, toUpper(replace(row.relation,' ', '_')),
  {},
  {},
  t,
  {}
)
YIELD rel
RETURN distinct 'done' AS result;
"""